# 반도체 제조 공정 프로세스 Pass & Fail 수율 예측

## 데이터 소개 및 분석 프로세스 수립
사용 데이터 출처: https://www.kaggle.com/datasets/paresh2047/uci-semcom  


### 데이터 모양  
|컬럼|의미|
|--|--|
|Time|센서 계측 시간|
|0~589|590개의 센서값|
|Pass/Fail|반도체 Wafer 불량 여부 <br/> (-1: 불량이 아님 / 1: 불량임)|

### 데이터 전처리 과정  
반도체의 경우 불량인 반도체의 개수가 정상인 반도체보다 훨씬 적음  
이에 따라 Target Feature 불균형 문제가 발생하므로 이를 해소해야함  

### 데이터 정제 과정  
PCA를 통한 주성분 분석  
센서의 경우 이상치가 많이 발생하므로 이상치 제거 필요  